In [1]:
import ast
import glob
import warnings
from collections import defaultdict
from datetime import date

import numpy as np
import pandas as pd
import wandb

today = date.today()
api = wandb.Api()

# # Find all csv files in the current directory
csv_files = glob.glob("/home/lev/projects/TopoBenchmarkX/big_csv/*.csv")
# # Collect all the names of the csv files without the extension
csv_names = [csv_file[:-4] for csv_file in csv_files]
project_name = "TopoBenchmarkX_Hypergraph"  
user = "telyatnikov_sap"

if project_name not in csv_names:
    runs = api.runs(f"{user}/{project_name}")

    summary_list, config_list, name_list = [], [], []
    for run in runs:
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        config_list.append(
            {k: v for k, v in run.config.items() if not k.startswith("_")}
        )

        # .name is the human-readable name of the run.
        name_list.append(run.name)

    runs_df = pd.DataFrame(
        {"summary": summary_list, "config": config_list, "name": name_list}
    )

    runs_df.to_csv(f"{project_name}.csv")
else:
    runs_df = pd.read_csv(f"{project_name}.csv", index_col=0)

    for row in runs_df.iloc:
        row["summary"] = ast.literal_eval(row["summary"])
        row["config"] = ast.literal_eval(row["config"])


for row in runs_df.iloc:
    row["summary"].update(row["config"])

lst = [i["summary"] for i in runs_df.iloc]
df = pd.DataFrame.from_dict(lst)

df_init = df.copy()

# Get average epoch run time
df["epoch_run_time"] = df["_runtime"] / df["epoch"]

In [2]:
def normalize_column(df, column_to_normalize):
    # Use json_normalize to flatten the nested dictionaries into separate columns
    flattened_df = pd.json_normalize(df[column_to_normalize])
    # Rename columns to include 'nested_column' prefix
    flattened_df.columns = [
        f"{column_to_normalize}.{col}" for col in flattened_df.columns
    ]
    # Concatenate the flattened DataFrame with the original DataFrame
    result_df = pd.concat([df, flattened_df], axis=1)
    # Get new columns names
    new_columns = flattened_df.columns
    # Drop the original nested column if needed
    result_df.drop(column_to_normalize, axis=1, inplace=True)
    return result_df, new_columns


# Config columns to normalize
columns_to_normalize = ["model", "dataset", "callbacks", "paths"]

# Keep track of config columns added
config_columns = []
for column in columns_to_normalize:
    df, columns = normalize_column(df, column)
    config_columns.extend(columns)

## Select models that have finished the runs

## Workout us_demographic 

In [3]:
# For every rows where df['dataset.parameters.data_name'] == 'US-county-demos' extend the 'dataset.parameters.data_name' with dataset.parameters.task_variable 
# and set it to 'US-county-demos' + '-' + dataset.parameters.task_variable
df.loc[df['dataset.parameters.data_name'] == 'US-county-demos', 'dataset.parameters.data_name'] = df.loc[df['dataset.parameters.data_name'] == 'US-county-demos', 'dataset.parameters.data_name'] + '-' + df.loc[df['dataset.parameters.data_name'] == 'US-county-demos', 'dataset.parameters.task_variable']

In [4]:
# Print all columns 10 per line
for i in range(0, len(df.columns), 5):
    print(list(df.columns[i:i + 5]))


['_runtime', 'test/loss', 'train/loss', 'train/auroc', 'train/precision']
['_step', 'lr-Adam', 'val/loss', 'val/auroc', '_timestamp']
['test/auroc', 'trainer/global_step', 'epoch', '_wandb', 'val/recall']
['train/recall', 'test/precision', 'train/accuracy', 'test/recall', 'val/accuracy']
['test/accuracy', 'val/precision', 'seed', 'tags', 'extras']
['trainer', 'ckpt_path', 'task_name', 'model/params/total', 'model/params/trainable']
['model/params/non_trainable', 'test/mae', 'test/mse', 'train/mae', 'val/mse']
['train/mse', 'val/mae', 'epoch_run_time', 'model.compile', 'model._target_']
['model.model_name', 'model.model_domain', 'model.loss.task', 'model.loss._target_', 'model.loss.loss_type']
['model.readout._target_', 'model.readout.hidden_dim', 'model.readout.readout_name', 'model.readout.num_cell_dimensions', 'model.backbone.beta']
['model.backbone.alpha', 'model.backbone._target_', 'model.backbone.n_layers', 'model.backbone.input_drop', 'model.backbone.layer_drop']
['model.backbone

### See unique datasets

In [5]:
print(df['dataset.parameters.data_name'].unique())
print("Num unique datasets:", len(df['dataset.parameters.data_name'].unique()))

['REDDIT-BINARY' 'minesweeper' 'tolokers' 'amazon_ratings' 'roman_empire'
 'IMDB-MULTI' 'IMDB-BINARY' 'NCI109' 'NCI1' 'PROTEINS' 'MUTAG' 'ZINC'
 'PubMed' 'citeseer' 'Cora' 'US-county-demos-UnemploymentRate'
 'US-county-demos-BachelorRate' 'US-county-demos-DeathRate'
 'US-county-demos-BirthRate' 'US-county-demos-MigraRate'
 'US-county-demos-MedianIncome' 'US-county-demos-Election']
Num unique datasets: 22


## See unique models

In [6]:
print(df['model.model_name'].unique())

['unignn2' 'edgnn' 'allsettransformer']


In [7]:
datasets = ['minesweeper', 'questions', 'tolokers', 'amazon_ratings', 'roman_empire']
models = ['unignn2', 'allsettransformer', 'edgnn']
# For the following models and datasets I mistook the batch size, it should be 1, instead of 256 or 128
# Keep the run where batch size is 128 and then change the batch size to 1
for model in models:
    print("MODEL:", model)
    for dataset in datasets:
        # Change the batch size to 1 when it is 128
        print(f"Dataset: {dataset} ", df.loc[(df['model.model_name'] == model) & (df['dataset.parameters.data_name'] == dataset), 'dataset.parameters.batch_size'].unique())
        
        
        


MODEL: unignn2
Dataset: minesweeper  [1]
Dataset: questions  []
Dataset: tolokers  [1]
Dataset: amazon_ratings  [1]
Dataset: roman_empire  [1]
MODEL: allsettransformer
Dataset: minesweeper  [1]
Dataset: questions  []
Dataset: tolokers  [1]
Dataset: amazon_ratings  [1]
Dataset: roman_empire  [1]
MODEL: edgnn
Dataset: minesweeper  [128 256]
Dataset: questions  []
Dataset: tolokers  [128 256]
Dataset: amazon_ratings  [128 256]
Dataset: roman_empire  [128 256]


## Solve batch problems

In [8]:
datasets = ['minesweeper', 'questions', 'tolokers', 'amazon_ratings', 'roman_empire']
models = ['edgnn']
# For the following models and datasets I mistook the batch size, it should be 1, instead of 256 or 128
# Keep the run where batch size is 128 and then change the batch size to 1
for model in models:
    for dataset in datasets:
        # Change the batch size to 1 when it is 128
        df.loc[(df['model.model_name'] == model) & (df['dataset.parameters.data_name'] == dataset) & (df['dataset.parameters.batch_size'] == 128), 'dataset.parameters.batch_size'] = 1
        # Drop runs where batch size is 256
        df.drop(df[(df['model.model_name'] == model) & (df['dataset.parameters.data_name'] == dataset) & (df['dataset.parameters.batch_size'] == 256)].index, inplace=True)
        

In [9]:
len(df.loc[(df['model.model_name'] == 'edgnn') & (df['dataset.parameters.data_name'] == 'minesweeper') & (df['dataset.parameters.batch_size'] == 128), 'dataset.parameters.batch_size'])

0

## Solve issue with projection dropout

In [10]:
print(df['model.feature_encoder.proj_dropout'].unique())

[0.5  0.25]


In [11]:
# Keep rows where model.feature_encoder.proj_dropout is [0.5  0.25]
df = df[df['model.feature_encoder.proj_dropout'].isin([0.5, 0.25])]


In [12]:
df.reset_index(drop=True, inplace=True)

In [13]:
# # Sweeped parameters: 
# sweeped_columns = [
#     'model.optimizer.lr', 
#     'model.feature_encoder.out_channels',
#     'model.backbone.n_layers',
#     'model.backbone.All_num_layers',
#     'model.feature_encoder.proj_dropout',
#     'dataset.parameters.batch_size',
#     'dataset.parameters.data_seed',
#     'seed',
# ]



# # For each model and dataset go over all the sweeped parameters and print the unique values
# for model in df['model.model_name'].unique():
#     print(f"Model: {model}")
#     for dataset in df['dataset.parameters.data_name'].unique():
#         print(f"Dataset: {dataset}")
#         for column in sweeped_columns:
#             print(f"Column: {column}")
#             print(df.loc[(df['model.model_name'] == model) & (df['dataset.parameters.data_name'] == dataset), column].unique())
        
#         print('---------------NEW DATASET------------------')
#     print('---------------NEW MODEL------------------')


### Get the best results

In [14]:
# Extract best results for each model and dataset
# 1. Keep the columns that are necessary for the comparison
sweeped_columns = [
    'model.optimizer.lr', 
    'model.feature_encoder.out_channels',
    'model.backbone.n_layers',
    'model.backbone.All_num_layers',
    'model.feature_encoder.proj_dropout',
    'dataset.parameters.batch_size',
    'model/params/total',
    # 'dataset.parameters.data_seed',
    # 'seed',
]
run_columns = ['dataset.parameters.data_seed','seed']

# Dataset and model columns
dataset_model_columns = ['model.model_name', 'dataset.parameters.data_name']

# Performance columns
performance_columns = [
    'val/loss', 'test/loss',
    'val/mae', 'test/mae',
    'val/mse', 'test/mse',
    'val/accuracy', 'test/accuracy',
    'val/auroc','test/auroc',
    'val/recall', 'test/recall',
    'val/precision', 'test/precision',
    ]
keep_columns = dataset_model_columns + sweeped_columns + performance_columns + run_columns
df = df[keep_columns]

In [15]:
performance_classification = [
    'val/accuracy', 'test/accuracy',
    'val/auroc','test/auroc',
    'val/recall', 'test/recall',
    'val/precision', 'test/precision',
    ]
performance_regression = [
    'val/mae', 'test/mae',
    'val/mse', 'test/mse',
    ]
# Define a dict of dicts for each dataset the corresponding optimization metrics
optimization_metrics = {
    'IMDB-MULTI': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'IMDB-BINARY': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'REDDIT-BINARY': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'NCI109': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'NCI1': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'PROTEINS': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'MUTAG': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'Cora': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'citeseer': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'PubMed': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},

    'roman_empire': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    'amazon_ratings': {'optim_metric': 'val/accuracy', 'eval_metric': 'test/accuracy', 'direction': 'max', 'performance_columns': performance_classification},
    
    'tolokers': {'optim_metric': 'val/auroc', 'eval_metric': 'test/auroc', 'direction': 'max', 'performance_columns': performance_classification},
    'questions': {'optim_metric': 'val/auroc', 'eval_metric': 'test/auroc', 'direction': 'max', 'performance_columns': performance_classification},
    'minesweeper': {'optim_metric': 'val/auroc', 'eval_metric': 'test/auroc', 'direction': 'max', 'performance_columns': performance_classification},

    'ZINC': {'optim_metric': 'val/mae', 'eval_metric': 'test/mae', 'direction': 'min', 'performance_columns': performance_regression},
    
    'US-county-demos-UnemploymentRate': {'optim_metric': 'val/mse', 'eval_metric': 'test/mse', 'direction': 'min', 'performance_columns': performance_regression},
    'US-county-demos-BachelorRate': {'optim_metric': 'val/mse', 'eval_metric': 'test/mse', 'direction': 'min', 'performance_columns': performance_regression},
    'US-county-demos-DeathRate': {'optim_metric': 'val/mse', 'eval_metric': 'test/mse', 'direction': 'min', 'performance_columns': performance_regression},
    'US-county-demos-BirthRate': {'optim_metric': 'val/mse', 'eval_metric': 'test/mse', 'direction': 'min', 'performance_columns': performance_regression},
    'US-county-demos-MigraRate': {'optim_metric': 'val/mse', 'eval_metric': 'test/mse', 'direction': 'min', 'performance_columns': performance_regression},
    'US-county-demos-MedianIncome': {'optim_metric': 'val/mse', 'eval_metric': 'test/mse', 'direction': 'min', 'performance_columns': performance_regression},
    'US-county-demos-Election': {'optim_metric': 'val/mse', 'eval_metric': 'test/mse', 'direction': 'min', 'performance_columns': performance_regression},

} 

len(optimization_metrics)

23

### Generate the best results

In [16]:
# Get unique datasets
datasets = list(df['dataset.parameters.data_name'].unique())
# Get unique models
models = list(df['model.model_name'].unique())

best_results = defaultdict(dict)
best_results_all_metrics = defaultdict(dict)
best_runs = defaultdict(dict)
collect_bast_parameters = defaultdict(dict)
# Got over each dataset and model and find the best result
for dataset in datasets:
    for model in models:
        # Get the subset of the DataFrame for the current dataset and model
        subset = df[
            (df['dataset.parameters.data_name'] == dataset)
            & (df['model.model_name'] == model)
        ]

        optim_metric = optimization_metrics[dataset]['optim_metric']
        eval_metric = optimization_metrics[dataset]['eval_metric']
        direction = optimization_metrics[dataset]['direction']
        
        # Keep metrics that matters for dataset
        performance_columns = optimization_metrics[dataset]['performance_columns']
        subset = subset[dataset_model_columns + sweeped_columns + performance_columns + run_columns]

        aggregated = subset.groupby(sweeped_columns, dropna=False).agg(
            {col: ["mean", "std"] for col in performance_columns}
        )

         # Go from MultiIndex to Index
        aggregated = aggregated.reset_index()
        aggregated = aggregated.sort_values(
                by=(optim_metric, "mean"), ascending=(direction == 'min')
            )
        
        # Git percent in case of classification
        if 'test/accuracy' in performance_columns:
            # Go over all the performance columns and multiply by 100
            for col in performance_columns:
                aggregated[(col, "mean")] *= 100
                aggregated[(col, "std")] *= 100
            
            # Round performance columns values up to 2 decimal points
            for col in performance_columns:
                aggregated[(col, "mean")] = aggregated[(col, "mean")].round(2)
                aggregated[(col, "std")] = aggregated[(col, "std")].round(2)
            
            
        else:
            # Round all values up to 4 decimal points
            # Round performance columns values up to 4 decimal points
            for col in performance_columns:
                aggregated[(col, "mean")] = aggregated[(col, "mean")].round(4)
                aggregated[(col, "std")] = aggregated[(col, "std")].round(4)
        
            
        
        # Get the best result
        final_best = aggregated.head(1)
        if final_best[(eval_metric, "mean")].any(): 
            best_results[dataset][model] = {
                "mean": final_best[(eval_metric, "mean")].values[0],
                "std": final_best[(eval_metric, "std")].values[0],
            }

            # Extract best runs: 
            best_params = {}
            for col in sweeped_columns:
                best_params[col] = final_best[(col, '')].item()
            
            collect_bast_parameters[dataset][model] = best_params
            #hp_runs[dataset][model] = subset.copy()
            
            # Start with the entire DataFrame
            filtered_subset = subset.copy()

            # Iterate over each key-value pair in the best parameters dictionary and filter the DataFrame
            for param, value in best_params.items():
                filtered_subset = filtered_subset[filtered_subset[param] == value]
            best_runs[dataset][model] = filtered_subset
        
        else: 
            best_results[dataset][model] = {
                "mean": np.nan,
                "std": np.nan,
            }
        
        
        


## Save obtained best results and best runs

In [17]:
# Convert nested dictionary to DataFrame
nested_dict = dict(best_results)
result_dict = pd.DataFrame.from_dict(
    {
        (i, j): nested_dict[i][j]
        for i in nested_dict
        for j in nested_dict[i].keys()
    },
    orient="index",
)

result_dict["performance"] = result_dict.apply(
    lambda x: f"{x['mean']} ± {x['std']}", axis=1
)
result_dict = result_dict.drop(["mean", "std"], axis=1)

# Reset multiindex
result_dict = result_dict.reset_index()
# rename columns
result_dict.columns = ["Dataset", "Model", "Performance"]

result_dict = result_dict.pivot_table(
    index="Model", columns="Dataset", values="Performance", aggfunc="first"
)

In [18]:
# Increase the number of allowed rows to display
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 1000)
result_dict.to_csv(f"best_results_hypergraph.csv")

In [19]:
result_dict

Dataset,Cora,IMDB-BINARY,IMDB-MULTI,MUTAG,NCI1,NCI109,PROTEINS,PubMed,REDDIT-BINARY,US-county-demos-BachelorRate,US-county-demos-BirthRate,US-county-demos-DeathRate,US-county-demos-Election,US-county-demos-MedianIncome,US-county-demos-MigraRate,US-county-demos-UnemploymentRate,ZINC,amazon_ratings,citeseer,minesweeper,roman_empire,tolokers
Model,,,,,,,,,,,,,,,,,,,,,,
allsettransformer,88.92 ± 0.44,70.32 ± 3.27,50.51 ± 2.92,71.06 ± 6.49,75.18 ± 1.24,73.75 ± 1.09,76.63 ± 1.74,89.62 ± 0.25,74.84 ± 2.68,0.2996 ± 0.0276,0.7111 ± 0.0823,0.4938 ± 0.0456,0.2897 ± 0.0136,0.2066 ± 0.0216,0.7775 ± 0.123,0.2199 ± 0.022,0.59 ± 0.0233,50.5 ± 0.27,73.85 ± 2.21,81.14 ± 0.05,79.5 ± 0.13,83.26 ± 0.1
edgnn,87.06 ± 1.09,69.12 ± 2.92,49.17 ± 4.35,80.0 ± 4.9,73.97 ± 0.82,74.93 ± 2.5,73.91 ± 4.39,89.04 ± 0.51,83.24 ± 1.45,0.2934 ± 0.0242,0.7036 ± 0.0743,0.5192 ± 0.0466,0.3418 ± 0.0246,0.225 ± 0.0236,0.7988 ± 0.1225,0.2576 ± 0.0281,0.5109 ± 0.011,48.18 ± 0.09,74.93 ± 1.39,84.52 ± 0.05,81.01 ± 0.24,77.53 ± 0.01
unignn2,86.97 ± 0.88,71.04 ± 1.31,49.76 ± 3.55,80.43 ± 4.09,73.02 ± 0.92,70.76 ± 1.11,75.2 ± 2.96,89.34 ± 0.45,75.56 ± 3.19,0.311 ± 0.0229,0.7257 ± 0.0952,0.511 ± 0.0454,0.367 ± 0.0204,0.2342 ± 0.0218,0.7923 ± 0.1162,0.2833 ± 0.0207,0.5991 ± 0.0056,49.06 ± 0.08,74.72 ± 1.08,78.02 ± 0.0,77.06 ± 0.2,77.35 ± 0.03


### Save best parameters

In [20]:
domain = "hypergraph"
# Convert nested dictionary to DataFrame
nested_dict = dict(collect_bast_parameters)
result_dict = pd.DataFrame.from_dict(
    {
        (i, j): nested_dict[i][j]
        for i in nested_dict
        for j in nested_dict[i].keys()
    },
    orient="index",
)


result_dict = result_dict.reset_index()

rename_cols = {
#'model.optimizer.lr': "lr",
#'model.feature_encoder.out_channels':"out_channels",
#'model.backbone.num_layers': 'num_layers',
#'model.feature_encoder.proj_dropout': 'proj_dropout',
#'dataset.parameters.batch_size': 'batch_size',
'level_0': 'Dataset', 'level_1': 'Model', 'model/params/total': 'Num. parameters'}

result_dict.rename(columns=rename_cols, inplace=True)
result_dict['Num. parameters'] = (result_dict['Num. parameters'] /1000).round(2).apply(lambda x: f"{x}K")
result_dict.to_csv(f"best_parameters_{domain}.csv")

result_dict = result_dict.pivot_table(
    index="Model", columns="Dataset", values='Num. parameters', aggfunc="first"
)
result_dict.to_csv(f"model_sizes_{domain}.csv")

In [21]:
result_dict

Dataset,Cora,IMDB-BINARY,IMDB-MULTI,MUTAG,NCI1,NCI109,PROTEINS,PubMed,REDDIT-BINARY,US-county-demos-BachelorRate,US-county-demos-BirthRate,US-county-demos-DeathRate,US-county-demos-Election,US-county-demos-MedianIncome,US-county-demos-MigraRate,US-county-demos-UnemploymentRate,ZINC,amazon_ratings,citeseer,minesweeper,roman_empire,tolokers
Model,,,,,,,,,,,,,,,,,,,,,,
allsettransformer,60.26K,114.24K,111.3K,80.77K,57.47K,221.57K,14.34K,280.83K,106.18K,316.93K,30.21K,316.93K,217.34K,217.34K,80.64K,105.86K,106.82K,155.91K,132.87K,118.02K,257.17K,217.99K
edgnn,113.29K,9.86K,27.01K,5.73K,88.19K,88.32K,5.6K,147.59K,5.83K,84.1K,84.1K,84.1K,21.57K,21.57K,21.57K,21.57K,22.53K,122.24K,258.5K,21.7K,41.49K,21.89K
unignn2,109.06K,100.61K,8.32K,84.1K,104.32K,4.61K,21.31K,114.56K,68.1K,3.55K,13.25K,51.07K,4.58K,4.58K,51.07K,5.6K,102.14K,89.22K,541.32K,51.33K,90.9K,13.57K
